In [ ]:
import pandas as pd, numpy as np, pickle
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf

# Load dataset
df = pd.read_csv('news.csv')
X = df['headline']
y = df['category']

# TF-IDF vectorization
vec = TfidfVectorizer(max_features=3000)
Xv = vec.fit_transform(X).toarray()

# Encode labels
le = LabelEncoder()
y_enc = le.fit_transform(y)

# Train-Test split
Xtr, Xte, ytr, yte = train_test_split(Xv, y_enc, test_size=0.2, random_state=42)

# Neural Network model
model = tf.keras.Sequential([
    tf.keras.layers.Dense(256, activation='relu', input_shape=(3000,)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(len(le.classes_), activation='softmax')
])

model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

model.fit(Xtr, ytr, validation_data=(Xte, yte), epochs=10)

# Save model and preprocessing objects
model.save('news_model.h5')
pickle.dump(vec, open('vectorizer.pkl', 'wb'))
pickle.dump(le.classes_, open('labels.pkl', 'wb'))

print('Training complete. Model and vectorizer saved!')
